In [1]:
import pandas as pd
import pandasql as ps
import numpy as np

In [2]:
field_playersDf = pd.read_csv('./Field_player_nodes.csv')
goal_keepersDf = pd.read_csv('./GK_nodes.csv')
hostsDf = pd.read_csv('./Hosts_edges.csv')
plays_forDf = pd.read_csv('./Plays_for_edges.csv')
teamsDf = pd.read_csv('./Team_nodes.csv')

In [4]:
field_playersDf.columns

Index(['player_optaId', 'postGame_subbedOut', 'postGame_RedCard',
       'postGame_minPlayed', 'preGame_side', 'preGame_line',
       'preGame_position', 'playerId', 'name', 'postGame_positions',
       'preGame_overall', 'preGame_potential', 'preGame_marketValueMilEuro',
       'preGame_preferredFoot', 'preGame_ageDays', 'postGame_error',
       'postGame_clearance', 'postGame_index', 'postGame_shots',
       'postGame_shots_on_target', 'postGame_shots_left_foot',
       'postGame_shots_right_foot', 'postGame_shots_head',
       'postGame_shots_other', 'postGame_goals', 'postGame_goals_left_foot',
       'postGame_goals_right_foot', 'postGame_goals_head',
       'postGame_goals_other', 'postGame_xG', 'postGame_cross',
       'postGame_cross_success', 'postGame_pass', 'postGame_pass_success',
       'postGame_pass_final_third', 'postGame_pass_final_third_success',
       'postGame_pass_forward', 'postGame_pass_forward_success',
       'postGame_dribble', 'postGame_dribble_success', 'po

In [13]:
ps.sqldf("""
    select name, player_optaId, postGame_shots as s, avg(postGame_shots) over(
        partition by player_optaId
        order by gameDate
        rows between 4 preceding
        and 1 preceding
    ) as avg_shots, strftime('%Y', gameDate) as year
    from field_playersDf
    where name = 'Lionel Messi'
    order by gameDate
    limit 20
""")

,name,player_optaId,s,avg_shots,year
0,Lionel Messi,19054,2,NaN,2018
1,Lionel Messi,19054,6,2.000000,2018
2,Lionel Messi,19054,2,4.000000,2018
3,Lionel Messi,19054,6,3.333333,2018
4,Lionel Messi,19054,2,4.000000,2018
5,Lionel Messi,19054,7,4.000000,2018
6,Lionel Messi,19054,4,4.250000,2018
7,Lionel Messi,19054,6,4.750000,2018
8,Lionel Messi,19054,5,4.750000,2018
9,Lionel Messi,19054,7,5.500000,2018


In [12]:
query = "select "
_columns = ['name', 'nodeName', 'player_optaId', 'postGame_subbedOut', 'postGame_RedCard',
       'postGame_minPlayed', 'preGame_side', 'preGame_line',
       'preGame_position', 'playerId', 'postGame_positions',
       'preGame_overall', 'preGame_potential', 'preGame_marketValueMilEuro',
       'preGame_preferredFoot', 'preGame_ageDays', 'gameId', 'preGame_xgpm', 'preGame_xppm', 'teamId',
       'teamName', 'gameDate', 'nodeId']
post_columns = ['postGame_error',
       'postGame_clearance', 'postGame_index', 'postGame_shots',
       'postGame_shots_on_target', 'postGame_shots_left_foot',
       'postGame_shots_right_foot', 'postGame_shots_head',
       'postGame_shots_other', 'postGame_goals', 'postGame_goals_left_foot',
       'postGame_goals_right_foot', 'postGame_goals_head',
       'postGame_goals_other', 'postGame_xG', 'postGame_cross',
       'postGame_cross_success', 'postGame_pass', 'postGame_pass_success',
       'postGame_pass_final_third', 'postGame_pass_final_third_success',
       'postGame_pass_forward', 'postGame_pass_forward_success',
       'postGame_dribble', 'postGame_dribble_success', 'postGame_tackle',
       'postGame_tackle_success', 'postGame_interception',
       'postGame_challenge', 'postGame_ball_recovery', 'postGame_ball_lost',
       'postGame_key_pass']
for column in _columns:
    q_plus = f"""
        {column},
    """
    query += q_plus
for i, column in enumerate(post_columns):
    try:
        post_columns[i+1]
        sep = ',' 
    except IndexError:
        sep = ''
    q_plus = f"""
                {column},

                avg({column}) over (
                    partition by player_optaId
                    order by gameDate
                    rows between 5 preceding
                    and 1 preceding
                ) 
                 as agg_{column}{sep}   
            """
    query += q_plus

query += """
    from field_playersDf outerFP
    order by gameDate, gameId, teamName
"""
features = ps.sqldf(query)
features.to_csv('field_players_plus_features.csv')